Libraries used for webscraping:
* Requests
* BeautifulSoup
* Pandas
* Re

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

Importing url of the British Airways Airline Reviews on airlineequality.com where the page display latest 1000 reviews

In [ ]:
base_url="https://www.airlinequality.com/airline-reviews/british-airways"
page_size=1000
url=f"{base_url}/page/1/?sortby=post_date%3ADesc&pagesize={page_size}/"

Parsing:
* Review Texts
* Flight Details
* Star Ratings



In [ ]:
response=requests.get(url)
content=response.content
parsed_content=BeautifulSoup(content,"html.parser")
reviews_whole=parsed_content.find_all("div",{"class":"text_content"})
flight_details_whole=parsed_content.find_all("table",{"class":"review-ratings"})
ratings_whole=parsed_content.find_all("span",{"class":"star fill"})

Scraping Reviews

In [ ]:
reviews=[]
print(f"Scrapping Page for Reviews")
for x in reviews_whole:
  reviews.append(x.get_text())
print(f"-->{len(reviews)} Total Reviews")
print(f"Scrapping Done")

Scrapping Page for Reviews
-->1000 Total Reviews
Scrapping Done


Scraping Flight Details

In [ ]:
flight_details=dict()
print(f"Scrapping Page for Flight Details")
for i,x in enumerate(flight_details_whole):
   flight_details[i]=x.get_text()
print(f"Scrapping Done")

Scrapping Page for Flight Details
Scrapping Done


Scraping Star Ratings

In [ ]:
ratings=dict()
pos=-1
print(f"Scrapping Page for Ratings")
for x in ratings_whole:
  if x.get_text()=='1':
    pos+=1
    ratings[pos]=x.get_text()
  else:
    ratings[pos]+=x.get_text()
print(f"Scrapping Done")

Scrapping Page for Ratings
Scrapping Done


Create a DataFrame to store scraped data

In [ ]:
df=pd.DataFrame()
df["Reviews"]=reviews

In [ ]:
column_list=["Aircraft","Type Of Traveller","Seat Type","Route","Date Flown","Seat Comfort","Cabin Staff Service","Food & Beverages","Inflight Entertainment","Ground Service","Wifi & Connectivity","Value For Money","Recommended"]
for x in column_list:
  df[x]=None

In [ ]:
text_column_list=["Aircraft","Type Of Traveller","Seat Type","Route","Date Flown","Recommended"]
star_column_list=["Seat Comfort","Cabin Staff Service","Food & Beverages","Inflight Entertainment","Ground Service","Wifi & Connectivity","Value For Money"]

In [ ]:
for i,x in flight_details.items():
  if i==0:
    continue
  else:
    split_list=x.split("\n")
    for y in text_column_list:
      pattern=f"{y}"
      for z in split_list:
        if(re.match(pattern,z)):
          temp=z.split(pattern)
          df.at[i-1,y]=temp[1]

In [ ]:
dfindex=0
scoreindex=5
for i,x in flight_details.items():
  if i==0:
    continue
  else:
    split_list=x.split("\n")
    for y in star_column_list:
      pattern=f"{y}"
      for z in split_list:
        if(re.match(pattern,z)):
          temp=len(ratings[scoreindex])
          df.at[dfindex,y]=temp
          scoreindex+=1
  dfindex+=1

Exporting DataFrame to a CSV File

In [ ]:
df.to_csv("reviews.csv")